## LangChain - Query translation

### Multi-Query

In [1]:
import bs4
from langchain import hub
from langchain.load import dumps, loads
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
## INDEXING ##

# Load blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)
# Make splits 
splits = text_splitter.split_documents(blog_docs)

# embedding
embedding = AzureOpenAIEmbeddings(
    model="text-embedding-3-large"
)

# Index
vector_store = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vector_store.as_retriever()

In [3]:
llm = AzureChatOpenAI(
    model="gpt-4o-mini",
    api_version="2024-12-01-preview"
)

In [4]:
## Prompt ##

# Multi Query: Different Perspectives
template = """
    You are an AI language model assistant. Your task is to generate five different
    versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is
    to help the user overcome some of the limitations of the distance-based similarity
    search. Provide these alternative questions separated by newlines.
    Original question: {question}
"""
prompt_perspective = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspective
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [5]:
def get_unique_union(documents: list[list]):
    """Unique union of retrieved docs"""
    # Flatten list of lists, and convert each Document to string
    flatten_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique document
    unique_docs = list(set(flatten_docs))
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})

C:\Users\BEU7CA\AppData\Local\Temp\ipykernel_10220\1830023186.py:7: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


In [6]:
len(docs)

6

In [7]:
from operator import itemgetter

template = """"
    Answer the following question based on this context:

    {context}
    Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {
        "context": retrieval_chain,
        "question": itemgetter("question")
    }
    | prompt
    | llm 
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents is the process of breaking down a complicated task into smaller, manageable subgoals. This approach enables efficient handling of complex tasks by allowing the agent to focus on individual components rather than tackling the entire problem at once. Techniques such as Chain of Thought (CoT) prompting help enhance model performance by instructing the model to think step by step, transforming large tasks into simpler steps. Tree of Thoughts (ToT) further extends this idea by exploring multiple reasoning possibilities for each step and creating a tree structure of thoughts to evaluate different paths. Task decomposition can be achieved through various methods, including simple prompting by the LLM, task-specific instructions, or human inputs.'